In [1]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import h5py
import nltk

Using TensorFlow backend.


In [2]:
### Import Packages
import pymongo
import re
import pandas as pd
from pprint import pprint

In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [4]:
db = client.local
### Emails is the collection name where you imported the csv file
collection = db.parsed_emails

In [26]:
cursors_list=collection.find({"x_folder":{"$regex":".*INBOX.*","$options": "-i"}}, {"message":1, "_id":0})

In [27]:
all_messages=[]
for cursor in cursors_list[:10000]:
    all_messages.append(cursor['message'])

In [26]:
cursors_list=collection.find({"x_folder":{"$regex":".*INBOX.*","$options": "-i"}}, {"subject":1, "_id":0})

In [ ]:
all_subjects=[]
unique_subjects=[]
for cursor in cursors_list[:10000]:
    all_subjects.append(cursor['subject'])
unique_subjects=all_subjects.unique

In [28]:
tokenized=[]
for message in all_messages:
    tokenized+=re.split('\W+|,| |\*| \n|\t|:|_', message)

In [36]:
tokens_file=open("/home/kavin/Silo/College Work/DS501/Case 5/10000_email_tokens.csv",'w')
for token in tokenized:
    if len(token) >0:
        tokens_file.write(token+"\n")
tokens_file.close()

In [22]:
tokens=pd.read_csv("/home/kavin/Silo/College Work/DS501/Case 5/10000_email_tokens.csv", names=("tokens",))


In [26]:
new_tokens=[]
for token in tokens[:100000].itertuples():
    #if(not str(token[1]).lower() in stopwords):
          new_tokens.append(str(token[1]).lower())
tokens=new_tokens

In [53]:
tokens

['please',
 'let',
 'know',
 'still',
 'need',
 'curve',
 'shift',
 'thanks',
 'heather',
 'original',
 'message',
 'allen',
 'phillip',
 'k',
 'sent',
 'friday',
 'december',
 '07',
 '2001',
 '5',
 '14',
 'dunton',
 'heather',
 'subject',
 'west',
 'position',
 'heather',
 'attach',
 'file',
 'email',
 'original',
 'message',
 'dunton',
 'heather',
 'sent',
 'wednesday',
 'december',
 '05',
 '2001',
 '1',
 '43',
 'pm',
 'allen',
 'phillip',
 'k',
 'belden',
 'tim',
 'subject',
 'fw',
 'west',
 'position',
 'attached',
 'delta',
 'position',
 '1',
 '16',
 '1',
 '30',
 '6',
 '19',
 '7',
 '13',
 '9',
 '21',
 'original',
 'message',
 'allen',
 'phillip',
 'k',
 'sent',
 'wednesday',
 'december',
 '05',
 '2001',
 '6',
 '41',
 'dunton',
 'heather',
 'subject',
 'west',
 'position',
 'heather',
 'exactly',
 'need',
 'would',
 'possible',
 'add',
 'prior',
 'day',
 'dates',
 'pivot',
 'table',
 'order',
 'validate',
 'curve',
 'shift',
 'dates',
 'also',
 'need',
 'prior',
 'days',
 'ending',

In [27]:
unique_words=pd.Series(tokens).unique()

In [6]:
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [7]:
stopwords=nltk.corpus.stopwords.words('english')

In [28]:
bag_of_words={}
index_to_words={}
index=0.0
for word in unique_words:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1
n_chars = len(tokens)
n_vocab = len(unique_words)

In [29]:
n_chars

100000

In [72]:
bag_of_words

{'20steve': 7068.0,
 'yellow': 1559.0,
 'four': 3410.0,
 'woods': 5885.0,
 'apital': 10217.0,
 'hanging': 2443.0,
 'woody': 5755.0,
 'looking': 1740.0,
 'eligible': 4946.0,
 'electricity': 1868.0,
 'unseating': 6793.0,
 'emptiv': 7425.0,
 '09phillip': 3143.0,
 'lora': 2468.0,
 'dela': 1883.0,
 'digit': 9940.0,
 'treasuries': 8256.0,
 'propane': 1998.0,
 'regional': 3379.0,
 'pierce': 6769.0,
 'bringing': 8386.0,
 'ometimes': 8219.0,
 'barraged': 8140.0,
 'basics': 2181.0,
 'internally': 5371.0,
 'prize': 373.0,
 'wednesday': 28.0,
 'satcher': 10576.0,
 'succession': 4159.0,
 'reliable': 4570.0,
 'nego': 9110.0,
 'expanded': 9387.0,
 'usbr': 4300.0,
 'quare': 10262.0,
 'miller': 3792.0,
 '270': 4973.0,
 'elegant': 1424.0,
 'second': 1829.0,
 '275': 1961.0,
 '277': 2730.0,
 'ifference': 8665.0,
 'errors': 8114.0,
 'relieving': 6384.0,
 'fingert': 1420.0,
 'cooking': 1339.0,
 'contributed': 6177.0,
 'carr': 117.0,
 'bonuses': 9355.0,
 '100012': 3001.0,
 'cars': 6714.0,
 'databank': 9935.0

In [30]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = tokens[i:i + seq_length]
	seq_out = tokens[i + seq_length]
	dataX.append([bag_of_words[word] for word in seq_in])
	dataY.append(bag_of_words[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  99900


In [31]:
len(dataX)

99900

In [32]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [33]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [76]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=2, batch_size=128, callbacks=callbacks_list)

Epoch 1/2
65989/65989 [==============================] - 275s - loss: 8.2081   
Epoch 2/2
65989/65989 [==============================] - 276s - loss: 7.9906   


In [34]:
# load the network weights
filename = "weights-improvement-12-2.0852.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

ValueError: Dimension 1 in both shapes must be equal, but are 10936 and 65 for 'Assign_5' (op: 'Assign') with input shapes: [256,10936], [256,65].

In [78]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ' '.join([index_to_words[value] for value in pattern]), "\""

Seed:
" commitments 20 move underscored tumultuous conditions sweeping houston energy trading concern past 10 days enron natio n largest energy trader principal nearly one quarter e lectricity natural gas trades yesterday example enron involv ed 4 billion deals enrononline unit 20 since early last week enron share price plummeted 50 last week reported 618 million third quarter loss reduction shareholder equity 1 2 billion also disclosed securities exchange commission conducting inquiry billions dollars transaction entities connected former chief financial officer rew fastow replaced wednesday 20 draw credit facilities came one rating agency fitch pu enron review possible downgrade another standard poor "


In [79]:
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = index_to_words[index]
	seq_in = [index_to_words[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

enronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenronenron